# Opioids Project

Ra'Kira Nelson and Alexa Fahrer

In [171]:
import numpy as np
import pandas as pd

pd.set_option("mode.copy_on_write", True)

## Prescriptions

In [172]:
prescriptions_raw = pd.read_parquet("data/ids590_opioids_by_drug_county_year.parquet")

In [173]:
prescriptions = prescriptions_raw.copy()
prescriptions

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm
0,AK,ANCHORAGE,2006,BUPRENORPHINE,30.0,125.154336
1,AK,ANCHORAGE,2006,BUPRENORPHINE,75.0,0.216367
2,AK,ANCHORAGE,2006,CODEINE,0.15,13362.99003
3,AK,ANCHORAGE,2006,DIHYDROCODEINE,0.25,4.27264
4,AK,ANCHORAGE,2006,FENTANYL,100.0,476.051283
...,...,...,...,...,...,...
563626,WY,WESTON,2019,METHADONE,4.0,25.4904
563627,WY,WESTON,2019,MORPHINE,1.0,332.57952
563628,WY,WESTON,2019,OXYCODONE,1.5,621.053064
563629,WY,WESTON,2019,OXYMORPHONE,3.0,9.63468


In [174]:
## temporary exploration [Ra'Kira]
prescriptions["mme_conversion_factor"].dtype
prescriptions["calc_base_wt_in_gm"].dtype

prescriptions["buyer_state"].sample(20)

prescriptions[prescriptions["buyer_state"] == "VI"]

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm
525970,VI,SAINT CROIX,2006,BUPRENORPHINE,30.0,0.360102
525971,VI,SAINT CROIX,2006,CODEINE,0.15,2158.08483
525972,VI,SAINT CROIX,2006,FENTANYL,100.0,7.56975
525973,VI,SAINT CROIX,2006,FENTANYL,130.0,0.900001
525974,VI,SAINT CROIX,2006,HYDROCODONE,1.0,767.489682
...,...,...,...,...,...,...
526380,VI,SAINT THOMAS,2019,HYDROCODONE,1.0,463.290826
526381,VI,SAINT THOMAS,2019,HYDROMORPHONE,4.0,34.35575
526382,VI,SAINT THOMAS,2019,MEPERIDINE,0.1,104.035313
526383,VI,SAINT THOMAS,2019,MORPHINE,1.0,138.116832


In [175]:
prescriptions["mme_conversion_factor"] = (
    prescriptions["mme_conversion_factor"].to_numpy().astype("float64")
)
prescriptions["calc_base_wt_in_gm"] = (
    prescriptions["calc_base_wt_in_gm"].to_numpy().astype("float64")
)
prescriptions["buyer_county"] = prescriptions["buyer_county"].str.upper().str.strip()
prescriptions["buyer_state"] = prescriptions["buyer_state"].str.upper().str.strip()

prescriptions = prescriptions[
    ~prescriptions["buyer_state"].isin(["PR", "VI", "GU", "MP", "AS", "PW"])
]

> why are we removing these states?

In [176]:
prescriptions

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm
0,AK,ANCHORAGE,2006,BUPRENORPHINE,30.00,125.154336
1,AK,ANCHORAGE,2006,BUPRENORPHINE,75.00,0.216367
2,AK,ANCHORAGE,2006,CODEINE,0.15,13362.990030
3,AK,ANCHORAGE,2006,DIHYDROCODEINE,0.25,4.272640
4,AK,ANCHORAGE,2006,FENTANYL,100.00,476.051283
...,...,...,...,...,...,...
563626,WY,WESTON,2019,METHADONE,4.00,25.490400
563627,WY,WESTON,2019,MORPHINE,1.00,332.579520
563628,WY,WESTON,2019,OXYCODONE,1.50,621.053064
563629,WY,WESTON,2019,OXYMORPHONE,3.00,9.634680


## Deaths

In [177]:
deaths_dfs = {}
for year in range(2003, 2016):
    key = f"deaths_{year}"
    url = (
        "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/"
        f"refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%20{year}.txt"
    )

    df = pd.read_csv(url, sep="\t", skipfooter=15, engine="python")
    df = df.drop(columns=["Notes"])
    deaths_dfs[key] = df

deaths = pd.concat(
    [deaths_dfs[f"deaths_{year}"].assign(year=year) for year in range(2006, 2016)],
    ignore_index=True,
)

In [178]:
# temp Ra'Kira chunk
deaths

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,year
0,"Autauga County, AL",1001,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,449.0,2006
1,"Baldwin County, AL",1003,2006.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,2006
2,"Baldwin County, AL",1003,2006.0,2006.0,All other alcohol-induced causes,A9,15.0,2006
3,"Baldwin County, AL",1003,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,1613.0,2006
4,"Barbour County, AL",1005,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,284.0,2006
...,...,...,...,...,...,...,...,...
44812,"Sweetwater County, WY",56037,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,251,2015
44813,"Teton County, WY",56039,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,95,2015
44814,"Uinta County, WY",56041,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,142,2015
44815,"Washakie County, WY",56043,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,81,2015


In [179]:
deaths["Year"] = pd.to_numeric(deaths["Year"], errors="coerce").astype("Int64")
# deaths[["County Name", "State"]] = deaths["County"].str.split(",", n=1, expand=True)
# deaths["County Name"] = deaths["County Name"].str.strip()
# deaths["State"] = deaths["State"].str.strip()
# deaths["County Name"] = (
#    deaths["County Name"].str.replace(r"\bCounty\b", "", regex=True).str.strip()
# )
deaths = deaths[
    deaths["Drug/Alcohol Induced Cause"]
    == "Drug poisonings (overdose) Unintentional (X40-X44)"
]
deaths["fips"] = deaths["County Code"].astype(str).str.zfill(5)
deaths = deaths.drop(columns=["Year Code", "year", "County Code"])

In [180]:
deaths

# deaths[deaths["County"].str.contains("aint")]

,County,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,fips
1,"Baldwin County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,01003
12,"Chilton County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,13.0,01021
39,"Jefferson County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,55.0,01073
55,"Mobile County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,23.0,01097
60,"Montgomery County, AL",2006,Drug poisonings (overdose) Unintentional (X40-...,D1,12.0,01101
...,...,...,...,...,...,...
44778,"Waukesha County, WI",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,34,55133
44784,"Winnebago County, WI",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,22,55139
44794,"Fremont County, WY",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,10,56013
44800,"Laramie County, WY",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,13,56021


## FIPS

In [181]:
fips = pd.read_excel("data/US_FIPS_Codes.xls", skiprows=1)

In [182]:
# temp RN code chunk
fips

,State,County Name,FIPS State,FIPS County
0,Alabama,Autauga,1,1
1,Alabama,Baldwin,1,3
2,Alabama,Barbour,1,5
3,Alabama,Bibb,1,7
4,Alabama,Blount,1,9
...,...,...,...,...
3137,Wyoming,Sweetwater,56,37
3138,Wyoming,Teton,56,39
3139,Wyoming,Uinta,56,41
3140,Wyoming,Washakie,56,43


In [183]:
fips["fips"] = fips["FIPS State"].astype(str).str.zfill(2) + fips["FIPS County"].astype(
    str
).str.zfill(3)

us_state_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "District of Columbia": "DC",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
}

fips["state_abbrev"] = fips["State"].map(us_state_abbrev)
fips["County Name"] = fips["County Name"].str.upper().str.strip()
fips["state_abbrev"] = fips["state_abbrev"].str.upper().str.strip()

fips["County Name"] = (
    fips["County Name"]
    .str.upper()
    .str.strip()
    .str.replace(r"^ST[.\s]+", "SAINT ", regex=True)
)

fips

,State,County Name,FIPS State,FIPS County,fips,state_abbrev
0,Alabama,AUTAUGA,1,1,01001,AL
1,Alabama,BALDWIN,1,3,01003,AL
2,Alabama,BARBOUR,1,5,01005,AL
3,Alabama,BIBB,1,7,01007,AL
4,Alabama,BLOUNT,1,9,01009,AL
...,...,...,...,...,...,...
3137,Wyoming,SWEETWATER,56,37,56037,WY
3138,Wyoming,TETON,56,39,56039,WY
3139,Wyoming,UINTA,56,41,56041,WY
3140,Wyoming,WASHAKIE,56,43,56043,WY


In [184]:
# temp RN code chunk
# add assert later
fips[fips["fips"].isnull()]

,State,County Name,FIPS State,FIPS County,fips,state_abbrev


In [185]:
prescriptions["buyer_county"] = prescriptions["buyer_county"].replace(
    {
        # Alaska
        "PETERSBURG": "WRANGELL PETERSBURG",
        "PRINCE OF WALES HYDER": "PRINCE WALES KETCHIKAN",
        "SKAGWAY": "SKAGWAY HOONAH ANGOON",
        "WRANGELL": "WRANGELL PETERSBURG",
        # Georgia / Illinois
        "DEKALB": "DE KALB",
        # Illinois
        "DUPAGE": "DU PAGE",
        # Indiana
        "ST JOSEPH": "SAINT JOSEPH",
        # Louisiana
        "ST JOHN THE BAPTIST": "SAINT JOHN THE BAPTIST",
        # Missouri
        "SAINTE GENEVIEVE": "STE GENEVIEVE",
        # Mississippi
        "DESOTO": "DE SOTO",
        # Virginia
        "SALEM": "SALEM CITY",
    }
)

In [186]:
# additional manipulations

print(fips[fips["County Name"].str.contains("KALB")])
print(fips[fips["County Name"].str.contains("SALEM")])

prescriptions.loc[
    (prescriptions["buyer_state"] == "MO")
    & (prescriptions["buyer_county"] == "DE KALB"),
    "buyer_county",
] = "DEKALB"

prescriptions.loc[
    (prescriptions["buyer_state"] == "TN")
    & (prescriptions["buyer_county"] == "DE KALB"),
    "buyer_county",
] = "DEKALB"

prescriptions.loc[
    (prescriptions["buyer_state"] == "NJ")
    & (prescriptions["buyer_county"] == "SALEM CITY"),
    "buyer_county",
] = "SALEM"

          State County Name  FIPS State  FIPS County   fips state_abbrev
24      Alabama     DE KALB           1           49  01049           AL
428     Georgia     DE KALB          13           89  13089           GA
610    Illinois     DE KALB          17           37  17037           IL
710     Indiana     DE KALB          18           33  18033           IN
1511   Missouri      DEKALB          29           63  29063           MO
2445  Tennessee      DEKALB          47           41  47041           TN
           State County Name  FIPS State  FIPS County   fips state_abbrev
1787  New Jersey       SALEM          34           33  34033           NJ
2945    Virginia  SALEM CITY          51          775  51775           VA


In [187]:
prescriptions_fips_merged = pd.merge(
    prescriptions,
    fips[["state_abbrev", "County Name", "fips"]],
    left_on=["buyer_state", "buyer_county"],
    right_on=["state_abbrev", "County Name"],
    how="left",
    indicator=True,
    validate="m:1",
)
prescriptions_fips_merged

prescriptions_merged = prescriptions_fips_merged.drop(
    columns=["state_abbrev", "County Name", "_merge"]
).copy()

In [188]:
print(prescriptions_fips_merged["_merge"].value_counts())

_merge
both          553750
left_only        147
right_only         0
Name: count, dtype: int64


In [189]:
unmatched = prescriptions_fips_merged[
    prescriptions_fips_merged["_merge"] == "left_only"
]
unmatched[["buyer_state", "buyer_county"]].drop_duplicates().sort_values(
    ["buyer_state", "buyer_county"]
)

,buyer_state,buyer_county
24515,AR,MONTGOMERY


In [190]:
# there is no MONTGOMERY, AR in fips df to merge with
# maybe there was a name change. can explore later

fips[fips["County Name"].str.contains("MONTGOMERY")]

,State,County Name,FIPS State,FIPS County,fips,state_abbrev
50,Alabama,MONTGOMERY,1,101,01101,AL
317,District of Columbia,MONTGOMERY,11,31,11031,DC
487,Georgia,MONTGOMERY,13,209,13209,GA
659,Illinois,MONTGOMERY,17,135,17135,IL
747,Indiana,MONTGOMERY,18,107,18107,IN
854,Iowa,MONTGOMERY,19,137,19137,IA
947,Kansas,MONTGOMERY,20,125,20125,KS
1076,Kentucky,MONTGOMERY,21,173,21173,KY
1204,Maryland,MONTGOMERY,24,31,24031,MD
1446,Mississippi,MONTGOMERY,28,97,28097,MS


## Population

In [191]:
pop_1 = pd.read_csv("data/co-est00int-tot.csv", encoding="latin1")
pop_2 = pd.read_csv("data/co-est2020.csv", encoding="latin1")

In [192]:
p1_sub = pop_1[
    [
        "STATE",
        "COUNTY",
        "STNAME",
        "CTYNAME",
        "POPESTIMATE2006",
        "POPESTIMATE2007",
        "POPESTIMATE2008",
        "POPESTIMATE2009",
    ]
].copy()
p2_sub = pop_2[
    [
        "STATE",
        "COUNTY",
        "STNAME",
        "CTYNAME",
        "POPESTIMATE2010",
        "POPESTIMATE2011",
        "POPESTIMATE2012",
        "POPESTIMATE2013",
        "POPESTIMATE2014",
        "POPESTIMATE2015",
    ]
].copy()
pop_merged = p1_sub.merge(
    p2_sub, on=["STATE", "COUNTY", "STNAME", "CTYNAME"], how="inner"
)
pop_merged["fips"] = pop_merged["STATE"].astype(str).str.zfill(2) + pop_merged[
    "COUNTY"
].astype(str).str.zfill(3)

In [193]:
pop_merged

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,fips
0,1,0,Alabama,Alabama,4628981,4672840,4718206,4757938,4785514,4799642,4816632,4831586,4843737,4854803,01000
1,1,1,Alabama,Autauga County,51328,52405,53277,54135,54761,55229,54970,54747,54922,54903,01001
2,1,3,Alabama,Baldwin County,168121,172404,175827,179406,183121,186579,190203,194978,199306,203101,01003
3,1,5,Alabama,Barbour County,27861,27757,27808,27657,27325,27344,27172,26946,26768,26300,01005
4,1,7,Alabama,Bibb County,22099,22438,22705,22941,22858,22736,22657,22510,22541,22553,01007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,56,37,Wyoming,Sweetwater County,39749,41470,42358,44133,43580,44000,45032,45189,44996,44780,56037
3184,56,39,Wyoming,Teton County,20014,20472,20988,21232,21298,21422,21643,22335,22801,23083,56039
3185,56,41,Wyoming,Uinta County,19709,20171,20613,21054,21090,20901,21008,20969,20835,20777,56041
3186,56,43,Wyoming,Washakie County,7979,8169,8229,8423,8531,8451,8410,8417,8277,8282,56043


In [194]:
year_cols = [c for c in pop_merged.columns if c.startswith("POPESTIMATE")]
pop_long = pop_merged.melt(
    id_vars=["fips", "STNAME", "CTYNAME"],
    value_vars=year_cols,
    var_name="pop_var",
    value_name="population",
)
pop_long["Year"] = pop_long["pop_var"].str.extract(r"(\d{4})").astype(int)
pop_long = pop_long.drop(columns=["pop_var"])
pop_long

,fips,STNAME,CTYNAME,population,Year
0,01000,Alabama,Alabama,4628981,2006
1,01001,Alabama,Autauga County,51328,2006
2,01003,Alabama,Baldwin County,168121,2006
3,01005,Alabama,Barbour County,27861,2006
4,01007,Alabama,Bibb County,22099,2006
...,...,...,...,...,...
31875,56037,Wyoming,Sweetwater County,44780,2015
31876,56039,Wyoming,Teton County,23083,2015
31877,56041,Wyoming,Uinta County,20777,2015
31878,56043,Wyoming,Washakie County,8282,2015


## Merging

In [ ]:
deaths_with_pop = pd.merge(
    deaths,
    pop_long[["fips", "Year", "population"]],
    on=["fips", "Year"],
    how="inner",  # changed here
    validate="m:1",
    indicator=True,
)
print(deaths_with_pop["_merge"].value_counts())

"""
changed from left to inner since all the rows that didn't join (left-only) 
did not all had missing num death values which won't be relevant to us
"""

# deaths_with_pop[deaths_with_pop["_merge"] != "both"]

_merge
both          6324
left_only        5
right_only       0
Name: count, dtype: int64


,County,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,fips,population,_merge
5568,"Prince of Wales-Outer Ketchikan Census Area, AK",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,Missing,02201,NaN,left_only
5569,"Skagway-Hoonah-Angoon Census Area, AK",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,Missing,02232,NaN,left_only
5570,"Wrangell-Petersburg Census Area, AK",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,Missing,02280,NaN,left_only
6259,"Bedford city, VA",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,Missing,51515,NaN,left_only
6261,"Clifton Forge city, VA",2015,Drug poisonings (overdose) Unintentional (X40-...,D1,Missing,51560,NaN,left_only


In [247]:
prescriptions_with_pop = pd.merge(
    prescriptions_merged,
    pop_long[["fips", "Year", "population"]].rename(columns={"Year": "year"}),
    on=["fips", "year"],
    how="left",
    validate="m:1",
    indicator=True,
)

print(prescriptions_with_pop["_merge"].value_counts())

# remove the years that aren't relevant to us
prescriptions_with_pop = prescriptions_with_pop[
    (prescriptions_with_pop["year"] < 2016) & (prescriptions_with_pop["year"] > 2005)
]

print(prescriptions_with_pop["_merge"].value_counts())

_merge
both          397131
left_only     156766
right_only         0
Name: count, dtype: int64
_merge
both          397131
left_only        608
right_only         0
Name: count, dtype: int64


## Current Dataframes

In [ ]:
prescriptions_with_pop = prescriptions_with_pop.drop(columns="_merge")
deaths_with_pop = deaths_with_pop.drop(columns=["_merge"])

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm,fips,population
0,AK,ANCHORAGE,2006,BUPRENORPHINE,30.00,125.154336,02020,280085.0
1,AK,ANCHORAGE,2006,BUPRENORPHINE,75.00,0.216367,02020,280085.0
2,AK,ANCHORAGE,2006,CODEINE,0.15,13362.990030,02020,280085.0
3,AK,ANCHORAGE,2006,DIHYDROCODEINE,0.25,4.272640,02020,280085.0
4,AK,ANCHORAGE,2006,FENTANYL,100.00,476.051283,02020,280085.0
...,...,...,...,...,...,...,...,...
553846,WY,WESTON,2015,METHADONE,4.00,49.192000,56045,7202.0
553847,WY,WESTON,2015,MORPHINE,1.00,680.033600,56045,7202.0
553848,WY,WESTON,2015,OXYCODONE,1.50,1168.363625,56045,7202.0
553849,WY,WESTON,2015,OXYMORPHONE,3.00,14.095180,56045,7202.0
